<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 要用process，不能用thread (要kill才不會影響其他System_UI)

1. 使用stop_event (check_stop) 在 各step function & run_report_function  (直接放在主程式了)


In [ ]:
import dash
from dash import html, dcc, Input, Output, State, no_update
import time
import threading


# 步驟定義
step_names = [
    "撈取資料",
    "進行 Rule Check",
    "繪製圖片",
    "上拋資料"
]

# 進度狀態
def default_progress():
    progress = {
        'current_index': -1,
        'step_status': ['pending'] * len(step_names),
        'step_times': [None] * len(step_names),
        'step_addition': [None] * len(step_names),
        'done': False,
        'error': False
    }
    return progress


# 檢查是否應停止
def check_stop():
    if stop_event.is_set():
        raise Exception("中止流程")

# 各步驟函式
def step1():
    check_stop()
    time.sleep(5)
    return 'aa'

def step2(x):
    check_stop()
    time.sleep(10)
    return 'aa', 'bb'

def step3(x,y):
    check_stop()
    time.sleep(3)
    return 'cc'

def step4(z):
    check_stop()
    time.sleep(3)
    return 'dd', 'z', 'aa'


# 執行流程的主函式 (各step function & 主function 都要添加 check_stop())
def run_report_process():
    try:
        check_stop()
        start_time = time.time()
        # step 1
        progress['current_index'] = 0
        progress['step_status'][0] = 'running'
        a = step1()
        check_stop()
        t1 = time.time()
        progress['step_times'][0] = int(t1-start_time)
        progress['step_addition'][0] = len(a)
        progress['step_status'][:2] = ['success','running']

        # step 2
        progress['current_index'] = 1
        b, c = step2(a)
        t2 = time.time()
        progress['step_times'][1] = int(t2-t1)
        progress['step_status'][1:3] = ['success','running']

        # step 3
        check_stop()
        progress['current_index'] = 2
        d = step3(b, c)
        check_stop()
        t3 = time.time()
        progress['step_times'][2] = int(t3-t2)
        progress['step_status'][2:4] = ['success','running']

        # step 4
        check_stop()
        progress['current_index'] = 3
        e = step4(d)
        check_stop()
        progress['step_times'][3] = int(time.time()-t3)
        progress['step_status'][3] = 'success'
        progress['done'] = True

        # 寄信 通知

    except Exception as e:
        progress['step_status'][progress['current_index']] = 'error'
        progress['error'] = True


1. 添加 kill btn
2. 開啟 新thread時， 先 stop_event.clear() (確保解除event觸發狀態)
3. (Callback 1) start report btn → (i) 出現狀態列，(ii) 開啟 interval，(iii)report btn 關閉
4. (Callback 2) start report btn → 確保舊thread關閉，開啟新thread

4. kill 按鈕 → (i) 隱藏狀態列，(ii) 關閉 interval，(iii)report btn 開啟
5. progress = done/error 或是 kill btn觸發 → 關閉interval。
6. 完成後寄信
  

In [ ]:
app = dash.Dash(__name__)
server = app.server

# Layout
app.layout = html.Div([
    html.H3("報告流程 "),
    html.Button("Report", id="run-btn", n_clicks=0),
    html.Button("Kill", id="kill-btn", n_clicks=0, style={"marginLeft": "10px"}),
    html.Div(id="step-list", style={'display':'none'}),
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=True)
])

## ======== start / stop process ========= ##
stop_event = threading.Event()  # 創建事件
current_thread = None
progress = default_progress()

# 啟動流程 (report btn → (i)interval.disabled=False, (ii) step-list 開啟, (iii)report btn.disabled=True)
@app.callback(
    Output("run-btn", "disabled"),
    Output('interval', "disabled"),
    Output('step-list', 'style'),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_process(n_clicks):
    if n_clicks>0:
        return True, False, {"marginTop": "20px", "fontSize": "16px"}
    else:
        return no_update, no_update, no_update

@app.callback(
    Output("run-btn", "style"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_process_thread(n_clicks):
    global current_thread, progress
    print(1)
    if current_thread and current_thread.is_alive():
        print(2)
        current_thread.join()

    # 重設狀態
    print(3)
    stop_event.clear()
    progress = default_progress()
    current_thread = threading.Thread(target=run_report_process)
    current_thread.start()
    return no_update

# 停止流程 ((i)關閉step-list, (ii)report btn.disabled=False, (iii)interval.disabled=True)
@app.callback(
    Output('step-list', 'style', allow_duplicate=True),
    Output("run-btn", "disabled", allow_duplicate=True),
    Output('interval', "disabled", allow_duplicate=True),
    Input("kill-btn", "n_clicks"),
    prevent_initial_call=True
)
def stop_process(n_clicks):
    global progress
    stop_event.set()  # 通知舊流程停止
    # 重設 未停止的舊thread 狀態 (若下一次start report按下時，舊thread還未停止 就show這個)
    progress = default_progress()
    progress['step_status'][0] = 'wating'
    progress['step_addition'][0] = '... wating the last progress stop'
    return {'display':'none'}, False, True


## ======== update UI ========= ##
@app.callback(
    Output("step-list", "children"),
    Output('interval', "disabled", allow_duplicate=True),
    Input("interval", "n_intervals"),
    prevent_initial_call=True
)
def update_ui(n):
    elements = []
    for i, step in enumerate(step_names):
        status = progress['step_status'][i]
        # dict.get(key, default_value) : 用來安全地取得鍵對應的值，即使該鍵不存在也不會報錯。
        color = {'pending': 'gray', 'running': 'orange', 'success': 'green', 'error': 'red','wating':'blue'}.get(status, 'gray')
        symbol = {'pending': "□", 'running': "▶", 'success': "✔", 'error': "❌", 'wating': "▶"}.get(status, "□")
        time_spent = f" ({progress['step_times'][i]}秒)" if progress['step_times'][i] else ""
        text_addition = f"({progress['step_addition'][i]})" if progress['step_addition'][i] else '' # "{}"裡可以放['']，'{}'不行
        elements.append(html.Div(f"{symbol} {step}{time_spent} {text_addition}", style={"color": color, "marginBottom": "5px"}))

    interval_stop = True if progress['done']==True or progress['error']==True else False

    return elements, interval_stop

if __name__ == "__main__":
    app.run(debug=True)